In [2]:
!pip install -U scikit-learn
!pip install torch
!pip install pytorch-transformers
!pip install accelerate -U
!pip install transformers


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from sagemaker import get_execution_role #this contains the permissions to access the bucket
import boto3 #aws python client
import pandas as pd
s3 = boto3.client('s3') #connect to S3
bucketName = "cs539-e2023-149b5d57-81a4-4e31-a476-c81e7e07dc33"

In [4]:
try:
    import sagemaker_datawrangler
except ImportError:
    !pip install --upgrade sagemaker-datawrangler
    import sagemaker_datawrangler

  Using cached sagemaker_datawrangler-0.4.3-py3-none-any.whl (10.6 MB)
  Using cached sagemaker_data_insights-0.4.0-py3-none-any.whl (99 kB)
  Using cached ipywidgets-7.7.5-py2.py3-none-any.whl (123 kB)
  Using cached sagemaker_scikit_learn_extension-2.5.0-py3-none-any.whl
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
  Using cached widgetsnbextension-3.6.4-py2.py3-none-any.whl (1.6 MB)
  Using cached jupyterlab_widgets-1.1.4-py3-none-any.whl (246 kB)
  Using cached notebook-7.0.0-py3-none-any.whl (4.0 MB)
  Using cached jupyter_server-2.7.0-py3-none-any.whl (375 kB)
  Using cached jupyterlab_server-2.24.0-py3-none-any.whl (57 kB)
  Using cached jupyterlab-4.0.3-py3-none-any.whl (9.2 MB)
  Using cached notebook_shim-0.2.3-py3-none-any.whl (13 kB)
  Using cached anyio-3.7.1-py3-none-any.whl (80 kB)
  Using cached argon2_cffi-21.3.0-py3-none-any.whl (14 kB)
  Using cached jupyter_events-0.6.3-py3-none-any.whl (18 kB)
  Using cached jupyter_server_terminals-0.4.4-py3-

In [11]:
def getChunkRange(start, end):
    df = None
    prefix = 'tokenized/tsv/training_reviews/'
    delim = 'chunk_*.tsv'
    #get all keys so we can skip ones that don't exist
    response = s3.list_objects_v2(Bucket=bucketName, Prefix=prefix, Delimiter=delim)['Contents']
    keys = []
    for obj in response:
        keys.append(obj['Key'])
    for chunk_id in range(start, end):
        chunk_key = prefix + 'chunk_' + '{:03d}'.format(chunk_id) + '.tsv'
        if chunk_key in keys:
            chunk_response = s3.get_object(Bucket=bucketName, Key=chunk_key)
            chunk_df = pd.read_csv(chunk_response.get('Body'), sep='\t')
            if df is None:
                df = chunk_df
            else:
                df = pd.concat([df,chunk_df], axis=0)
        else:
            print('WARNING: Chunk Key:', chunk_key, ' not found in bucket')
    return df

In [12]:
df = getChunkRange(0, 10)

In [13]:
def parse_string_to_list(value):
    cleaned_string = value.strip("[]").replace(" ", "")
    number_substrings = cleaned_string.split(',')
    integer_list = [int(number) for number in number_substrings]
    return integer_list

In [14]:
df['token_ids'] = df['token_ids'].map(parse_string_to_list)
df['attention_mask'] = df['attention_mask'].map(parse_string_to_list)

In [15]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, train_size=0.8, random_state=42, stratify=df['flagged'])

In [19]:
print(train_df['flagged'].value_counts()) 


flagged
0    46973
1     6642
Name: count, dtype: int64
flagged
0    11743
1     1661
Name: count, dtype: int64


In [11]:
from torch.utils.data import Dataset

class YelpDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

In [12]:
import torch
from torch.utils.data import DataLoader, TensorDataset
# Convert lists to tensors
train_input_ids = torch.tensor(train_df['token_ids'].tolist())
train_attention_masks = torch.tensor(train_df['attention_mask'].tolist())
train_labels = torch.tensor(train_df['flagged'].tolist())

test_input_ids = torch.tensor(test_df['token_ids'].tolist())
test_attention_masks = torch.tensor(test_df['attention_mask'].tolist())
test_labels = torch.tensor(test_df['flagged'].tolist())

# Create TensorDatasets for train and test sets
train_dataset = YelpDataset(train_input_ids, train_attention_masks, train_labels)
test_dataset = YelpDataset(test_input_ids, test_attention_masks, test_labels)


In [23]:
print(train_df['flagged'].value_counts())
print(train_df['flagged'].value_counts())

flagged
0    46973
1     6642
Name: count, dtype: int64
flagged
0    46973
1     6642
Name: count, dtype: int64


In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 

In [15]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)
import torch.nn as nn
import torch.nn.functional as F

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

# Define Trainer
args = TrainingArguments(
    output_dir="s3://cs539-e2023-149b5d57-81a4-4e31-a476-c81e7e07dc33/Model/",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=3,
    warmup_steps=500,
    adam_epsilon=1e-8,)

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=len(train_dataset)/args.per_device_train_batch_size * args.num_train_epochs)

optimizers = (optimizer,scheduler)
trainer = Trainer(
model=model,
args=args,
compute_metrics=compute_metrics,
train_dataset=train_dataset,
eval_dataset=test_dataset,
optimizers = optimizers,
)
# Train pre-trained model
trainer.train()

trainer.save_model('s3://cs539-e2023-149b5d57-81a4-4e31-a476-c81e7e07dc33/Model/')

# Evaluate the model on the test dataset
results = trainer.evaluate()

print("Results:", results)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.348700,0.326726,0.876679,1.000000,0.004816,0.009587
2,0.315000,0.319457,0.878395,0.556364,0.092113,0.158058
3,0.249300,0.344290,0.870263,0.442647,0.181216,0.257155


Results: {'eval_loss': 0.3442904055118561, 'eval_accuracy': 0.8702626081766637, 'eval_precision': 0.4426470588235294, 'eval_recall': 0.18121613485851895, 'eval_f1': 0.2571550619393422, 'eval_runtime': 212.2519, 'eval_samples_per_second': 63.151, 'eval_steps_per_second': 7.896, 'epoch': 3.0}


In [17]:
trainer.evaluate()

{'eval_loss': 0.3442904055118561,
 'eval_accuracy': 0.8702626081766637,
 'eval_precision': 0.4426470588235294,
 'eval_recall': 0.18121613485851895,
 'eval_f1': 0.2571550619393422,
 'eval_runtime': 210.42,
 'eval_samples_per_second': 63.701,
 'eval_steps_per_second': 7.965,
 'epoch': 3.0}

In [18]:
trainer.save_model('s3://cs539-e2023-149b5d57-81a4-4e31-a476-c81e7e07dc33/Model')

In [22]:
def getChunkRangeBig(start, end):
    df = None
    prefix = 'tokenized/tsv/reviews/'
    delim = 'chunk_*.tsv'
    #get all keys so we can skip ones that don't exist
    response = s3.list_objects_v2(Bucket=bucketName, Prefix=prefix, Delimiter=delim)['Contents']
    keys = []
    for obj in response:
        keys.append(obj['Key'])
    for chunk_id in range(start, end):
        chunk_key = prefix + 'chunk_' + '{:03d}'.format(chunk_id) + '.tsv'
        if chunk_key in keys:
            chunk_response = s3.get_object(Bucket=bucketName, Key=chunk_key)
            chunk_df = pd.read_csv(chunk_response.get('Body'), sep='\t')
            if df is None:
                df = chunk_df
            else:
                df = pd.concat([df,chunk_df], axis=0)
        else:
            print('WARNING: Chunk Key:', chunk_key, ' not found in bucket')
    return df

In [23]:
from torch.utils.data import Dataset

class YelpDatasetBig(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
        }

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset



df_big = getChunkRangeBig(0, 3)
df_big['token_ids'] = df_big['token_ids'].map(parse_string_to_list)
df_big['attention_mask'] = df_big['attention_mask'].map(parse_string_to_list)
df_big['labels'] = 0
# Convert lists to tensors
input_ids_big = torch.tensor(df_big['token_ids'].tolist())
attention_masks_big = torch.tensor(df_big['attention_mask'].tolist())
labels_big = torch.tensor(df_big['labels'].tolist())
# Create TensorDatasets for train and test sets
dataset_big = YelpDataset(input_ids_big, attention_masks_big, labels_big)

predictions = trainer.predict(dataset_big)

# Extract the predicted label IDs (class with highest probability)
predicted_label_ids = predictions.predictions.argmax(axis=1)

# Create a new DataFrame to store the results with 'reviewID', 'text', and 'predicted_label' columns
result_df = pd.DataFrame({'review_ids': df_big['review_id'], 'predicted_label': predicted_label_ids})
